In [1]:
%matplotlib inline

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

import RLConn as rc

W0410 15:17:43.494889 4610170304 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:5: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0410 15:17:43.516357 4610170304 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:58: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0410 15:17:43.523471 4610170304 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:66: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0410 15:17:43.527235 4610170304 deprecation.py:323] From /Users/Frank/github/RLConn/RLConn/control_dqn.py:68: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0410 15:17:44.035542 4610170304 deprecation_wrapper.py:119] From /Users/Frank/github/RLConn/RLConn/control

In [3]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [4]:
def get_observe(Gg, Gs):
    N = Gg.shape[0]
    return np.concatenate((Gg[np.triu_indices(N,k=1)],Gs[np.triu_indices(N,k=1)],Gs[np.tril_indices(N,k=-1)]))

In [5]:
def move(Gg, Gs, action, location, dx = 0.1):
    N = Gg.shape[0]
    size = int(N*(N-1)/2)
    change = dx*(action == 0) + (-dx)*(action == 2)
    if location < size:
        coordinate = np.array(np.triu_indices(N,k=1)).T[location]
        Gg[coordinate[0], coordinate[1]] += change
        Gg[coordinate[1], coordinate[0]] += change
    elif location < 2*size:
        coordinate = np.array(np.triu_indices(N,k=1)).T[location - size]
        Gs[coordinate[0], coordinate[1]] += change
    else: 
        coordinate = np.array(np.tril_indices(N,k=-1)).T[location - 2*size]
        Gs[coordinate[0], coordinate[1]] += change
    return Gg, Gs

In [6]:
def get_rand_action(N):
    size = int(N*(N-1)/2)
    return np.random.randint(2), np.random.randint(3*size)

In [7]:
from RLConn import problem_definitions as problems

def get_reward(Gg, Gs):
    with suppress_stdout():
        mean_err,_ = rc.utils.compute_problem_score(Gg, Gs, problems.FOUR_NEURON_OSCILLATION, verbose=False)
    np.random.seed() #reset seed
    return -mean_err

In [8]:
def update_x(x, Gg, Gs, reward):
    obs = get_observe(Gg, Gs)
    x = np.concatenate((x[len(obs)+1:], obs))
    x = np.append(x,reward)
    return x

In [9]:
# initialize
N = 4
size = int(3*N*(N-1)/2)
PG = rc.alpg.ActLocPolicyGradient(
    n_actions=3,
    n_features=(size+1)*10)

input_vec = np.zeros(4)
input_vec[1] = 0.068
ablation_mask = np.ones(4)

In [ ]:
for i_episode in range(3000):
    #initialize x
    x = np.array([])
    network = rc.network_sim.generate_random_network(4, 1, 15)
    Gg, Gs = network['gap'], network['syn']
    reward = get_reward(Gg, Gs)
    x = np.concatenate((x, get_observe(Gg, Gs)))
    x = np.append(x,reward)
    for i in range(9):
        action, location = get_rand_action(N)
        Gg, Gs = move(Gg, Gs, action, location, dx=1)
        reward = get_reward(Gg, Gs)
        x = np.concatenate((x, get_observe(Gg, Gs)))
        x = np.append(x,reward)   
        
    #choose action using nn
    for step in range(300):
        action, location = PG.choose_action(x)
        Gg, Gs = move(Gg, Gs, action, location, dx=1)
        reward = get_reward(Gg, Gs)
        PG.store_transition(x, action, location, reward)
        x = update_x(x, Gg, Gs, reward)
    ep_rs_sum = sum(PG.ep_rs)
    if 'running_reward' not in globals():
        running_reward = ep_rs_sum
    else:
        running_reward = running_reward * 0.99 + ep_rs_sum * 0.01
    print("episode:", i_episode, "  reward:", int(running_reward))
    print("Gg:", Gg.flatten(), "  Gs:", Gs.flatten())
    with suppress_stdout():
        mean_error, _ = rc.utils.compute_score(Gg, Gs, network['directionality'], 
                                                input_vec, ablation_mask,
                                                  tf = 7, t_delta = 0.01,
                                                  cutoff_1 = 100, cutoff_2 = 600,
                                                  plot_result = True)
    print(mean_error)
    vt = PG.learn()
    # print(vt)

/Users/Frank/miniconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/Frank/github/RLConn/RLConn/network_sim.py:535: RuntimeWarning: overflow encountered in true_divide
  J2 = np.multiply(Gsyn, J2_M4_2) / params_obj_neural['C']
/Users/Frank/miniconda3/lib/python3.7/site-packages/scipy/integrate/_ode.py:1009: UserWarning: vode: Repeated convergence failures. (Perhaps bad Jacobian supplied or wrong choice of MF or tolerances.)
  self.messages.get(istate, unexpected_istate_msg)))
/Users/Frank/github/RLConn/RLConn/network_sim.py:515: RuntimeWarning: overflow encountered in true_divide
  dV = (-(VsubEc + GapCon + SynapCon) + Input)/params_obj_neural['C']


episode: 0   reward: -59395
Gg: [  0 -14  13   0 -14   0   5 -11  13   5   0   1   0 -11   1   0]   Gs: [  0  13 -11  -1  10   0   5   0   1   2   0   8  13  13   6   0]
400.5466386471142
episode: 1   reward: -58855
Gg: [ 0  9  6 13  9  0 14 11  6 14  0 11 13 11 11  0]   Gs: [ 0  8 18  1 10  0  1  8  5 14  0  4  6 13  5  0]
17.883172803899498
